In [1]:
import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets.datasets import ATLASDataset 
from torchlib.neuralnet import NeuralNetClassifier
from misc import get_transforms_det

from sklearn import metrics

In [2]:
pathname = os.path.expanduser( '~/.datasets' )
projectnet='../out/netruns'
no_cuda=False
seed=1
gpu=0
batch_size=10
workers=0
num_input_channels=3
parallel=False

print(pathname)
print(projectnet)

/home/fernandez/.datasets
../out/netruns


In [3]:
!ls ../out/netruns/

atlas_baseline_preactresnet18_bcewl_adam_atlas_001
atlas_baseline_preactresnet18_mix_sgd_atlas_002
atlas_baseline_preactresnet18_mix_sgd_atlas_003
results


In [4]:
name = 'atlas_baseline_preactresnet18_mix_sgd_atlas_003'
project = os.path.join(projectnet, name)
model = 'model_best.pth.tar' #'chk000025.pth.tar', model_best
pathmodels = os.path.join(project,'models',model)

print(project)
print(pathmodels)

../out/netruns/atlas_baseline_preactresnet18_mix_sgd_atlas_003
../out/netruns/atlas_baseline_preactresnet18_mix_sgd_atlas_003/models/model_best.pth.tar


In [5]:
# neuralnet
network = NeuralNetClassifier(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu,
    parallel=parallel,
    )

cudnn.benchmark = True

# load model
if network.load( pathmodels ) is not True:
    assert(False)

Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loading checkpoint '../out/netruns/atlas_baseline_preactresnet18_mix_sgd_atlas_003/models/model_best.pth.tar'
=> loaded checkpoint for preactresnet18 arch!


In [7]:
pathdata='~/.kaggle/competitions/human-protein-atlas-image-classification'
name_dataset='atlas'
subset='test'

data = ATLASDataset(        
    path=pathdata, 
    train=False,
    folders_images='test', 
    metadata='sample_submission.csv',
    #count=100,
    num_channels=3,
    transform=get_transforms_det( network.size_input ), #get_transforms_aug
    )

dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )

print('Data loader ')
print(len(dataloader))
print(len(data))

Data loader 
1171
11702


In [8]:
Id, Yhat, Y = network.predict( dataloader )

df = pd.DataFrame( np.concatenate((Id[:,np.newaxis], Yhat), axis=1) )
df.to_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset,name,name_dataset)), index=False, encoding='utf-8')        
print('dir: {}'.format(project))
print('DONE!!!')

100%|██████████| 1171/1171 [04:29<00:00,  4.35it/s]


dir: ../out/netruns/atlas_baseline_preactresnet18_mix_sgd_atlas_003
DONE!!!


In [9]:
df = pd.read_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset, name, name_dataset))  )
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0.000152,0.000096,0.999998,0.083919,0.001511,0.000057,0.000954,0.002336,0.000255,...,0.000811,0.000414,0.000134,0.000044,0.000058,0.001155,0.000905,0.000232,0.000448,0.000014
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0.001973,0.000115,0.003231,0.015907,0.001066,0.979994,0.456646,0.005007,0.000676,...,0.003546,0.001079,0.001348,0.003201,0.000328,0.004170,0.004076,0.062390,0.001821,0.000252
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0.904326,0.000587,0.000774,0.001563,0.004422,0.987965,0.003545,0.001022,0.000191,...,0.002978,0.001089,0.000502,0.195035,0.003699,0.001288,0.000341,0.969727,0.001354,0.000140
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0.953833,0.004134,0.033613,0.006675,0.006259,0.015230,0.006001,0.021688,0.000499,...,0.007339,0.008444,0.001516,0.043042,0.001835,0.027898,0.000989,0.487863,0.004142,0.000370
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0.179600,0.002640,0.019394,0.006542,0.011531,0.019983,0.071884,0.080595,0.000731,...,0.004655,0.001712,0.001482,0.013135,0.000620,0.038863,0.000629,0.427773,0.002018,0.000289


In [10]:
result = df.as_matrix()
iDs = result[:,0]
Yhat = result[:,1:]

print(iDs.shape)
print(Yhat.shape)


(11702,)
(11702, 28)


In [11]:
(Yhat>0.4).sum(axis=0)

array([5096,  337,  781,  436,  345,  648,  329,  885,    0,    0,    0,
        276,  276,  125,  426,    0,    0,    0,   81,  402,    0, 1459,
         91,  513,   18, 4973,    0,    0])

In [84]:
pred = [ ' '.join( map(str,np.where( yhat > 0.4 )[0]) ) for yhat in  Yhat]
#code  = ' '.join( map(str, code) )
len(np.where( np.array([1,2,3]) < 0 )[0]) 

0

In [45]:
# submission_filepath = 'submission.csv'
# th=0.5
# #submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
# submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0]) ) }  for (Id, yhat) in zip( iDs,Yhat) ]
# submission = pd.DataFrame( submission )
# submission.to_csv(submission_filepath, index=False, encoding='utf-8')
# print('SAVE!!!')
# submission


In [12]:
submission_filepath = 'submission.csv'
th=0.4
submission = pd.read_csv('~/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')

#submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission_local = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0]) ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission_local = pd.DataFrame( submission_local )
submission.loc[ submission['Id'] == submission_local['Id']   , 'Predicted'] = submission_local['Predicted']
submission.to_csv(submission_filepath, index=False, encoding='utf-8')
submission


,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 6
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,25


In [18]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f submission.csv -m "preactresnet18+mix+256x256"

100%|█████████████████████████████████████████| 465k/465k [00:01<00:00, 473kB/s]
Successfully submitted to Human Protein Atlas Image Classification

In [24]:
submission = pd.read_csv('/root/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')
submission

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0
